In [1]:
"""
Builds the fundamental dataset for top 2000 market cap equitities from WRDS.
Requires WRDS account. Enter username and password when prompted.

Features: datadate,	gvkey,	year,  month,  mom1m,	mom3m,	mom6m,	mom9m,
        mrkcap,	entval,	saleq_ttm,	cogsq_ttm,	xsgaq_ttm,	oiadpq_ttm,
        niq_ttm,	cheq_mrq,	rectq_mrq,	invtq_mrq,	acoq_mrq,
        ppentq_mrq,	aoq_mrq,	dlcq_mrq,	apq_mrq,	txpq_mrq,
            lcoq_mrq,   ltq_mrq,	csho_1yr_avg,GSECTOR,GSUBIND 

Takes about 1.5 minutes to build the complete dataset and outputs a csv
"""

import wrds
import pandas as pd
import datetime
import numpy as np
import pickle
from time import time
from wrds_data_processing import data_processing
import sys

start_time = time()

# Connect to WRDS data engine
db = wrds.Connection()

#############################################################################
#### SQL Query-----------------------------------------------------------####
#############################################################################

# Query to get list of companies with top 20 market cap
q1 = ("select a.gvkey,a.latest,b.cshoq,b.prccq,b.mkvaltq,b.cshoq*b.prccq as market_cap,b.curcdq "
     "from "
        "(select gvkey,max(datadate) as latest "
         "from "
         "compm.fundq where datadate > '2017-01-01' "
         "group by gvkey) a inner join "
             "(select gvkey,datadate,mkvaltq,cshoq,prccq,curcdq "
                "from compm.fundq where cshoq>0 and prccq>0 and curcdq='USD') b "
    "on a.gvkey = b.gvkey and a.latest=b.datadate "
     "order by market_cap desc "
    "limit 20")

Enter your WRDS username [liqiran]:ansonccy
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [2]:
mrk_df = db.raw_sql(q1)
top_20_eq_gvkey_list = mrk_df['gvkey'].values.tolist()
top_20_eq_gvkey = tuple(["'%s'"%str(i) for i in top_20_eq_gvkey_list])
top_20_eq_gvkey = ",".join(top_20_eq_gvkey)


# Query to get fundamental Data
q2 = ("select datadate,gvkey,tic,saleq,cogsq,xsgaq,oiadpq,niq,revtq,oibdpq,xintq,nopiq,spiq,piq,txtq,ibq,epspxq,epsfxq,"
      "cheq, rectq, invtq, acoq, actq, ppentq, ivaeqq, ivaoq, intanq, aoq, atq, dlcq, apq, txpq,"
      "lcoq, lctq, dlttq, txditcq, loq, ltq, mibq,"
      "pstkq , ceqq, seqq, cshoq "
      "from compm.fundq "
      "where gvkey in (%s) ")%top_20_eq_gvkey
fundq_df = db.raw_sql(q2)

print("Shape of raw dataframe: %g,%g"%fundq_df.shape)
print('\n')

# Query to get price data
q3 = ("select gvkey,datadate,prccm "
     "from compm.secm "
     "where gvkey in (%s) ")%top_20_eq_gvkey
price_df_all = db.raw_sql(q3).sort_values('datadate')

# Query to get stock_split data
q4 = ("select gvkey,datadate,split "
     "from compm.sec_split "
     "where gvkey in (%s) ")%top_20_eq_gvkey
stock_split_df_all = db.raw_sql(q4).sort_values('datadate')
####--------------------------------------------------------------------------

Shape of raw dataframe: 2863,43




In [3]:
# Build balance sheet features
blnc_sheet_list = ['cheq','rectq','invtq','acoq','actq','ppentq','ivaeqq','ivaoq','intanq','aoq','atq',
                                'dlcq','apq','txpq','lcoq','lctq','dlttq','txditcq','loq','ltq','mibq',
                   'pstkq','ceqq','seqq','cshoq']

# Build income sheet features
income_list = ['saleq','cogsq','xsgaq','oiadpq','niq','revtq','xintq','nopiq','spiq',
               'piq','txtq','ibq','epspxq','epsfxq']


gvkey_list = top_20_eq_gvkey_list
print len(gvkey_list)

df_all = fundq_df[['gvkey','datadate'] + income_list + blnc_sheet_list]

def reorder_cols():
    a = ['datadate','gvkey','year','month']
    mom = ['mom1m','mom3m','mom6m','mom9m']
    prc = ['mrkcap','entval']
    ttm_list_tmp = [x + '_ttm' for x in income_list]
    mrq_list_tmp = [x + '_mrq' for x in blnc_sheet_list]
    mrq_list_tmp.remove('cshoq_mrq')
    mrq_list_tmp.remove('dlttq_mrq')
    csho = ['csho_1yr_avg']

    new_order = a + mom + prc + ttm_list_tmp + mrq_list_tmp + csho
    return new_order


20


In [4]:
# Create empty df to be appended for each equity
df_all_eq = pd.DataFrame(columns=reorder_cols())

run_time_month = datetime.datetime.now().date().month

for key in gvkey_list:
    #print("GVKEY: %s"%key)
    df = df_all[df_all['gvkey'] == key].copy()
    df = df.sort_values('datadate')
    df = df.set_index('datadate',drop=False)
    df = df[~df.index.duplicated(keep='first')]
    #print("df shape:%g,%g"%df.shape)
    

    # get price_df for the current gvkey
    price_df = price_df_all[price_df_all['gvkey']==key].copy()
    #print("price df shape:%g,%g"%price_df.shape)
    
    # get stock_split_df for the current gvkey
    stock_split_df = stock_split_df_all[stock_split_df_all['gvkey']==key].copy()
    #print("stock split df shape:%g,%g"%stock_split_df.shape)
    #print("\n")

    # Start data processing
    dp = data_processing(lag=3)
    
    # Add the lag to the date index
    df = dp.add_lag(df)
    
    
    # Create new df with monthly frequency (empty)
    new_df_empty = dp.create_df_monthly(df)
    
    # Add ttm and mrq data
    ttm_mrq_df = dp.create_ttm_mrq(df,new_df_empty)
    #print ttm_mrq_df.head()
    
    # Adjust for stock split
    df_split_adjusted = dp.adjust_cshoq(ttm_mrq_df,stock_split_df)

    # Add price information
    df_w_price,price_df_for_mom = dp.add_price_features(df_split_adjusted,price_df)
    
    # Add momentum features
    df_w_mom = dp.get_mom(df_w_price,price_df_for_mom,[1,3,6,9])
    #print df_w_mom.head()
    # Add csho_1_year average
    df_w_mom['csho_1yr_avg'] = df_w_mom['cshoq_mrq'].rolling(12,min_periods=1).mean()
    
    
    # Reorder column names
    new_order = reorder_cols()

    del df,price_df,stock_split_df

    df_out = df_w_mom[new_order]

    # Fill Nans with 0.0
    df_out = df_out.fillna(0.0)
    df_out = df_out.reset_index(drop=True)
    # Append the current df to the full_df
    df_all_eq = df_all_eq.append(df_out,ignore_index=True)
    if(run_time_month >= 1 and run_time_month <= 3):
        df_all_eq = df_all_eq.loc[df_all_eq['month'] == 12]
    elif (run_time_month > 3 and run_time_month <= 6):
        df_all_eq = df_all_eq.loc[df_all_eq['month'] == 3]
    elif (run_time_month > 6 and run_time_month <= 9):
        df_all_eq = df_all_eq.loc[df_all_eq['month'] == 6]
    elif (run_time_month > 9 and run_time_month <= 12):
        df_all_eq = df_all_eq.loc[df_all_eq['month'] == 9]
        

#

In [5]:
# Normalize the momentum features
dates = df_all_eq['datadate'].unique()
mom_f = ['mom1m','mom3m','mom6m','mom9m']


for date in dates:
    df_date = df_all_eq[mom_f][df_all_eq['datadate']==date]
    ix_dates = df_date.index
    df_norm = (df_date - df_date.min())/(df_date.max() - df_date.min())

    df_norm = df_norm.fillna(0.0)

    df_all_eq.loc[ix_dates,mom_f] = df_norm

    del df_date, df_norm

    #r = ['gvkey','datadate','fyear']
   #a =['aco','act','ao','aoloch','ap','apalch','aqc','at','capx','ceq','che'cogs	dlc	dlcch	dltis	dltr	dltt	dpc	dv	epsfx	epspx	esub	fiao	fincf	fopo	ib	ibc	intan	invch	invt	ivaco	ivaeq	ivao	ivch	ivncf	ivstch	lco	lct	lo	lt	mib	niadj	nopi	oancf	oiadp	oibdp	pi	ppent	prstkc	pstk	recch	rect	revt	seq	siv	spi	sppe	sppiv	sstk	txach	txbcof	txdc	txditc	txp	txt	xidoc	xint	xsga	costat

def rereorder_cols():
    a = ['datadate','gvkey','fyear','month']
    mom = ['mom1m','mom3m','mom6m','mom9m']
    prc = ['mrkcap','entval']
    ttm_list_tmp = ['sale','cogs','xsga','oiadp','ni','revt','xint','nopi','spi',
               'pi','txt','ib','epspx','epsfx']
    mrq_list_tmp = ['che','rect','invt','aco','act','ppent','ivaeq','ivao','intan','ao','at',
                                'dlc','ap','txp','lco','lct','dltt','txditc','lo','lt','mib',
                   'pstk','ceq','seq','csho']
    mrq_list_tmp.remove('csho')
    mrq_list_tmp.remove('dltt')
    csho = ['csho_1yr_avg']

    new_order = a + mom + prc + ttm_list_tmp + mrq_list_tmp + csho
    return new_order


df_all_eq.columns = rereorder_cols()
df_all_eq = df_all_eq.drop(['month','mom1m','mom3m','mom6m','mom9m','mrkcap','entval'], 1)


cols = df_all_eq.columns.tolist()

cols = ['gvkey','datadate','fyear','aco','act','ao','ap','at','ceq','che','cogs','dlc','epsfx','epspx','ib','intan','invt','ivaeq','ivao','lco','lct','lo','lt','mib','oiadp','pi','ppent','pstk','rect','revt','seq','spi','txditc','txp','txt','xint','xsga']
remain = ['csho_1yr_avg','ni','sale']
left_in_combined_less_impor = ['indfmt','consol','popsrc','datafmt','tic','curcd','costat']
left_in_combined_impor = ['aoloch','apalch','aqc','capx','dlcch','dltis','dltr','dltt','dpc','dv','esub','fiao','fincf','fopo','ibc','invch','ivaco','ivch','ivncf','ivstch','niadj','nopi','oancf','oibdp','prstkc','pstk','recch','siv','sppe','sppiv','sstk','txbcof','txdc','xidoc']
df_all_eq = df_all_eq[cols]


#not_in_compm = [curcd dltty niadjy pstky txbchy]
q5 = ("select gvkey, datadate, indfmt, consol, popsrc, datafmt, tic, costat,"
      "aolochy, apalchy, aqcy, capxy, dlcchy, dltisy, dltry, dpcy, dvy, esuby, "
      "fiaoy, fincfy, fopoy, ibcy, invchy, ivacoy, ivchy, ivncfy, ivstchy, nopiy, "
      "oancfy, oibdpy, prstkcy,  recchy, sivy, sppey, sppivy, sstky, txbcofy, txdcy, xidocy "
    "from compm.fundq "
    "where gvkey in (%s) ")%top_20_eq_gvkey
others = db.raw_sql(q5)
cols3 = ['gvkey', 'datadate','indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'costat',
      'aoloch', 'apalch', 'aqc', 'capx', 'dlcch', 'dltis', 'dltr', 'dpc', 'dv', 'esub',
      'fiao', 'fincf', 'fopo', 'ibc', 'invch', 'ivaco', 'ivch', 'ivncf', 'ivstch', 'nopi', 
      'oancf', 'oibdp', 'prstkc',  'recch', 'siv', 'sppe', 'sppiv', 'sstk', 'txbcof', 'txdc', 'xidoc']
#df1 = pd.DataFrame(columns=cols3)
others = others.drop(['gvkey', 'datadate'],1)


q6 = ("select gvkey, datadate, curcd, dltt, pstk, niadj,txach " 
      "from compa.funda "
     "where gvkey in (%s) ")%top_20_eq_gvkey
others2 = db.raw_sql(q6)
cols4 = ['gvkey', 'datadate', 'curcd', 'dltt', 'pstk', 'niadj','txach'] 
#df2 = pd.DataFrame(columns=cols4)
others2 = others2.drop(['gvkey', 'datadate'],1)

df_all_eq =  pd.concat([df_all_eq, others, others2], axis=1)


cols = ['gvkey','datadate','fyear','indfmt','consol','popsrc','datafmt','tic','curcd','aco','act','ao','aoloch','ap','apalch','aqc','at','capx','ceq','che','cogs','dlc','dlcch','dltis','dltr','dltt','dpc','dv','epsfx','epspx','esub','fiao','fincf','fopo','ib','ibc','intan','invch','invt','ivaco','ivaeq','ivao','ivch','ivncf','ivstch','lco','lct','lo','lt','mib','niadj','nopi','oancf','oiadp','oibdp','pi','ppent','prstkc','pstk','recch','rect','revt','seq','siv','spi','sppe','sppiv','sstk','txach','txbcof','txdc','txditc','txp','txt','xidoc','xint','xsga','costat']
cols2 = ['gvkey','datadate','fyear','aco','act','ao','ap','at','ceq','che','cogs','dlc','epsfx','epspx','ib','intan','invt','ivaeq','ivao','lco','lct','lo','lt','mib','oiadp','pi','ppent','pstk','rect','revt','seq','spi','txditc','txp','txt','xint','xsga',
        'indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'costat',
      'aoloch', 'apalch', 'aqc', 'capx', 'dlcch', 'dltis', 'dltr', 'dpc', 'dv', 'esub',
      'fiao', 'fincf', 'fopo', 'ibc', 'invch', 'ivaco', 'ivch', 'ivncf', 'ivstch', 'nopi', 
      'oancf', 'oibdp', 'prstkc',  'recch', 'siv', 'sppe', 'sppiv', 'sstk', 'txbcof', 'txdc', 'xidoc' ,'curcd', 'dltt', 'pstk', 'niadj','txach']

df_all_eq.columns = cols2;

df_all_eq = df_all_eq[cols]

def to_integer(dt_time):
    return 10000*dt_time.year + 100*dt_time.month + dt_time.day

df_all_eq['datadate'] = df_all_eq['datadate'].apply(to_integer)

df_all_eq = df_all_eq.fillna(0.0)
df_all_eq = df_all_eq.reset_index(drop=True)

print df_all_eq.head()
# Output the csv
df_all_eq.to_csv("top_20_eq_w_3mo_lag1.csv")
exec_time = time() -start_time

print exec_time


    gvkey    datadate   fyear indfmt consol popsrc datafmt   tic curcd  \
0  001690  19810301.0  1981.0   INDL      C      D     STD  AAPL   USD   
1  001690  19820301.0  1982.0   INDL      C      D     STD  AAPL   USD   
2  001690  19830301.0  1983.0   INDL      C      D     STD  AAPL   USD   
3  001690  19840301.0  1984.0   INDL      C      D     STD  AAPL   USD   
4  001690  19850301.0  1985.0   INDL      C      D     STD  AAPL   USD   

      aco   ...    txach  txbcof  txdc  txditc     txp     txt  xidoc   xint  \
0   0.076   ...      0.0     0.0   0.0   0.523   5.892  12.454    0.0  0.210   
1   0.504   ...      0.0     0.0   0.0   6.964   0.268  37.123    0.0  1.300   
2   8.067   ...      0.0     0.0   0.0  10.919  12.000  55.466    0.0  1.712   
3  23.334   ...      0.0     0.0   0.0  35.785   5.432  69.408    0.0  1.712   
4  50.735   ...      0.0     0.0   0.0  63.977  11.301  45.130    0.0  1.712   

      xsga  costat  
0   26.211       A  
1   88.279       A  
2  192.851 